In [1]:
import LiveAuctionProcessor
import PrevInfo
import importlib
import time
import psycopg2 as pg2
import pandas as pd
from datetime import datetime, timedelta
import math
import pickle

In [48]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

print ("Reading Dataset")
full_df = pd.read_sql ("""Select *  from auction_full where auctionid = 100022191""", conn)
bids_df = pd.read_sql("Select * from bids where auctionid = 100022191 ORDER BY bid", conn)
auctions_df = pd.read_sql("Select * from auctions where qauctionid = 100022191", conn)
bh = [{'bid':b[1]['bid'], 'username':b[1]['username'], 'is_bidomatic': b[1]['is_bidomatic']} for b in bids_df.iterrows()]

Connecting to SQL
Reading Dataset


In [32]:
#prev_df = pd.read_sql("""
#with bozo as
#(
#Select username, max(auctiontime) as max_auctiontime from auction_full WHERE auctiontime <= '2020-01-16 22:53:19' group by username 
#)
#select bozo.username,  prev_auction_count0, prev_overbid0, prev_giveup_one0, prev_give_before_six0, 
#prev_wins0, prev_bids0, prev_bom_bids0 from bozo left join auction_full on bozo.username=Username0 
#and bozo.max_auctiontime=auction_full.auctiontime and auction_full.bids_so_far0=1
#"""
#, conn)
pi = PrevInfo.PrevInfo()
pi.get_users_previous_info("07robert")
pi.pickle("pi.pkl")

Connecting to SQL


In [11]:
auction_id = '100022191'
importlib.reload(LiveAuctionProcessor)
auction_dict = {"auction_id":'620712999'}
lap = LiveAuctionProcessor.LiveAuctionProcessor(auction_dict)
bh = lap_update()

NameError: name 'lap_update' is not defined

In [128]:
#from IPython.display import clear_output

#while (True):
#    clear_output()
#    bh = lap.update()
#    for b in bh[-5:]:
#        print (b)
#    time.sleep(1)

In [49]:
bh

[{'bid': 1, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 2, 'username': 'Wiz1910', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 4, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 5, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 6, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 7, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 8, 'username': 'moon0218', 'is_bidomatic': False},
 {'bid': 9, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 10, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 11, 'username': 'XMAS19722014', 'is_bidomatic': False},
 {'bid': 12, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 13, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 14, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 15, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 16, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 17, 'usern

In [50]:
bh = bh[:7]

In [101]:
auction_dict = {"auctionid": full_df.auctionid[0],
  'cashvalue': full_df.cashvalue[0],
  'cardvalue': full_df.cardvalue[0],
  'cardtype': full_df.cardtype[0],
  'bidvalue': full_df.bidvalue[0],
  'limited_allowed': full_df.limited_allowed[0],
  'is_locked': False,    #FIXME
  'seconds_left': 531}


In [102]:
def add_auction_level_fields(auction_dict, my_username):
    auction_dict["idx"]=0
    auction_dict["is_winner"] = None
    auction_dict["auctiontime"] = datetime.now()+ timedelta(hours=1)
    bid = len(bh)+1
    auction_dict["bid"] = bid
    auction_dict["is_bidomatic"] = True
    auction_dict["bids_so_far"] = None
    auction_dict["username"] = my_username
    auction_dict["fee"]= None
    auction_dict["giveup"] = None
    auction_dict["eventual_bids"] = None
    auction_dict["eventual_win_price"] = None
    auction_dict["debut"] = None
    auction_dict["bom_streak"] = None
    auction_dict["bom_bids_so_far"] = None
    auction_dict["perc_to_bin"] = None

In [103]:
prev_cols = ['prev_auction_count', 'prev_overbid', 'prev_giveup_one', 'prev_give_before_six', 'prev_wins', 'prev_bids', 'prev_bom_bids']

In [107]:
my_username =  'moon0218'
add_auction_level_fields(auction_dict, my_username)
user_dict = {}
cur_opponent = 0
found_users = {}
for r in bh[-1::-1]:
    username = r['username']
    cur_bid = r['bid']
    is_bidomatic = r['is_bidomatic']
    if username == my_username:
        break
    if (username not in found_users.keys()):
        user_dict[cur_opponent]={"username":username, "bids_so_far": 0.0, "bom_bids_so_far": 0.0, "distance": float(bid-cur_bid), "is_bidomatic": is_bidomatic, "bom_streak": 0.0,  "bom_streak_on": True}
        found_users[username] = cur_opponent
        cur_opponent += 1
    my_row = user_dict[found_users[username]] 
    my_row ["bids_so_far"] += 1
    if (is_bidomatic):
        my_row["bom_bids_so_far"]+=1
        if my_row["bom_streak_on"]:
            my_row["bom_streak"] += 1
    else:
        my_row["bom_streak_on"] = False
        
prev_4_users_len = min(4, len(user_dict))    
for i in range(prev_4_users_len):
    str_i = str(i)
    ud = user_dict[i]
    prev_info = pi.get_users_previous_info(ud["username"])
    ud["prev_is_new_user"] = 0   #FixMe
    for pc in prev_cols:
        ud[pc] = prev_info[pc+"0"][prev_info.index[0]]
    #, prev_overbid0, prev_giveup_one0, prev_give_before_six0, prev_wins0, prev_bids0, prev_bom_bids0
    ud["perc_to_bin"]=ud["bids_so_far"]/(auction_dict["cashvalue"]*2.5)
    for k in ud:
        auction_dict[k+str_i] = ud[k]

for i in range (prev_4_users_len,4):
    for pc in prev_cols:
        ud[pc] = None
    
auction_dict["prevusers"] = len(user_dict)
auction_dict = {k:[v] for k, v in auction_dict.items()} 


In [108]:
auction_dict

{'auctionid': [100022191],
 'cashvalue': [10],
 'cardvalue': [0],
 'cardtype': ['None'],
 'bidvalue': [25],
 'limited_allowed': [False],
 'is_locked': [False],
 'seconds_left': [531],
 'idx': [0],
 'is_winner': [None],
 'auctiontime': [datetime.datetime(2020, 2, 29, 13, 51, 20, 292351)],
 'bid': [8],
 'is_bidomatic': [True],
 'bids_so_far': [None],
 'username': ['moon0218'],
 'fee': [None],
 'giveup': [None],
 'eventual_bids': [None],
 'eventual_win_price': [None],
 'debut': [None],
 'bom_streak': [None],
 'bom_bids_so_far': [None],
 'perc_to_bin': [None],
 'username0': ['mmmwww'],
 'bids_so_far0': [3.0],
 'bom_bids_so_far0': [0.0],
 'distance0': [1],
 'is_bidomatic0': [False],
 'bom_streak0': [0.0],
 'bom_streak_on0': [False],
 'prev_is_new_user0': [0],
 'prev_auction_count0': [616.0],
 'prev_overbid0': [0.16233766233766234],
 'prev_giveup_one0': [0.09253246753246754],
 'prev_give_before_six0': [0.22402597402597402],
 'prev_wins0': [0.22402597402597402],
 'prev_bids0': [54.50974025974

In [109]:

df_out = pd.DataFrame.from_dict(auction_dict)


In [110]:
my_row = df_out.iloc[0,:]


In [111]:
real_row = full_df.loc[full_df.bid == 8,:]

In [113]:
for c in real_row.columns:
    if (my_row[c] != real_row[c][22]):
        print(c, my_row[c], real_row[c][22])

is_winner None False
fee None 0.0
auctiontime 2020-02-29T13:51:20.292351000 2020-01-16 22:53:19
is_bidomatic True False
bids_so_far None 1
giveup None False
eventual_bids None 3
eventual_win_price None 40
debut None 1
bom_streak None 0
bom_bids_so_far None 0
perc_to_bin None 0.04


In [96]:
full_df.columns

Index(['auctionid', 'is_winner', 'cardtype', 'cashvalue', 'cardvalue', 'fee',
       'bidvalue', 'limited_allowed', 'is_locked', 'auctiontime', 'bid',
       'is_bidomatic', 'bids_so_far', 'username', 'prevusers', 'giveup',
       'eventual_bids', 'eventual_win_price', 'debut', 'bom_streak',
       'bom_bids_so_far', 'perc_to_bin', 'username0', 'distance0',
       'is_bidomatic0', 'bids_so_far0', 'bom_bids_so_far0', 'bom_streak0',
       'perc_to_bin0', 'prev_is_new_user0', 'prev_auction_count0',
       'prev_overbid0', 'prev_giveup_one0', 'prev_give_before_six0',
       'prev_wins0', 'prev_bids0', 'prev_bom_bids0', 'username1', 'distance1',
       'is_bidomatic1', 'bids_so_far1', 'bom_bids_so_far1', 'bom_streak1',
       'perc_to_bin1', 'prev_is_new_user1', 'prev_auction_count1',
       'prev_overbid1', 'prev_giveup_one1', 'prev_give_before_six1',
       'prev_wins1', 'prev_bids1', 'prev_bom_bids1', 'username2', 'distance2',
       'is_bidomatic2', 'bids_so_far2', 'bom_bids_so_far2', 

In [97]:
df_out = df_out[full_df.columns]

In [25]:
df_out

,auctionid,cashvalue,cardvalue,cardtype,bidvalue,limited_allowed,is_locked,seconds_left,idx,is_winner,...,prev_bom_bids1,perc_to_bin1,prevusers,giveup,eventual_bids,eventual_win_price,debut,bom_streak,bom_bids_so_far,perc_to_bin
0,100022191,10,0,None,25,False,False,531,0,None,...,15.621622,0.04,2,None,None,None,None,None,None,None


In [170]:
bh

[{'bid': 1, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 2, 'username': 'Wiz1910', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 4, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 5, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 6, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 7, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 8, 'username': 'moon0218', 'is_bidomatic': False},
 {'bid': 9, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 10, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 11, 'username': 'XMAS19722014', 'is_bidomatic': False},
 {'bid': 12, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 13, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 14, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 15, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 16, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 17, 'usern

In [278]:
 pm = pickle.load( open( "rf.pkl", "rb" ) )

Using TensorFlow backend.


In [285]:
pm.predict_proba(df_out)

2. Transforming data


array([[0.345, 0.655]])